#  <div align='center'> Ticket Analysis and guess SLA compliance </div>

> The code is to Analyse the tickets from customer and build the mode to guess is the SLA was met for a set of input provided or not  

Last Update: 28-Feb-2017
<br>



In [228]:
import pandas
import numpy as np

# Algorithms 
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier

# Other tools for model training and accuracy check
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score 


In [229]:
GE_Tkts = pandas.read_csv("GEW Inc1.csv")

In [230]:
# The columns we'll use to predict the target
 
# predictors = ["Grp Code", "Cat Code", "Sub Cat code", "Priority Code"]

predictors = [ "Cat Code", "Sub Cat code", "Priority Code"]

In [245]:
# Initialize our algorithm class
# alg = tree.DecisionTreeClassifier(min_samples_split=2, criterion="entropy", max_features = "auto")  # Around 72.81%
# alg = tree.DecisionTreeClassifier()  # Around 72.62%
# alg = SVC(C=2.0, kernel = "rbf") #kernel = "rbf" resulted = 73% 
# alg =  GaussianNB()
# alg =  BernoulliNB()
# alg = KNeighborsClassifier(algorithm="ball_tree", weights = "distance") #Around 71.8%
# alg = LinearRegression()
# alg = Lasso(alpha = .01)
# alg = AdaBoostClassifier(n_estimators = 3000) ## Acc = 72.42
# alg = GradientBoostingClassifier(n_estimators = 4000)
alg = RandomForestClassifier(criterion = "entropy")  #Best accuracy of 73.39%

## <div align= "center"> Analysis of accuracy with different parameters </div>

+ Working with SVC
> __Kernel = rbf__ ..
> Does good job with accuracy score of 72.02%. 
> When removed the predictor Grp Code, it improved to __72.62%__
> <br> *kernel* = poly took long time to complete and did not come back.
> <br> parameter value *C*=2 bumped the accuracy score from 72.62% to __73%__. Default C value is 1 (accu=72.02%)
> <br> Stack xchange [link]( http://stats.stackexchange.com/questions/31066/what-is-the-influence-of-c-in-svms-with-linear-kernel) about effect of value *C* in SVMs. Basically what should be the margin set during fitting. 
> <br> More detailed info is in [Sklearn documentation](http://scikit-learn.org/stable/auto_examples/svm/plot_rbf_parameters.html)
> <br> I tried many values, Default *gamma* was doing better job, 
+ Decision Trees
> Decition tree with default value resulted accuracy = 72.62%
> <br> with *min_samples_split*=2, *criterion*="entropy", *max_features* = "auto" -> Accuracy increased to = __72.81%__
+ KNeighborsClassifier
> Parameters = *algorithm*="auto", *weights* = "distance". 
> With an accuracy of 71.84%
> <br> When removed the Grp Code as predictor - it atuall decreased to 71.26%. 
> As far of *algorithm* parameter is concerned, auto and kd_tree has the same results while ball_tree reduced the accuracy a bit
+ Gaussian Naive Bayes
> This with default values performed poorly with accuracy of 67.76%
+ AdaBoost Classifier and GradientBoostingClassifier
> Default accuracy = 70.29%
> <br> with parameter *n_estimators* the accuracy impacted a lot. With the value *n_estimators* = 3000, the accuracy boosted to __72.42%__
> GradientBoost the accuracy was 72.23% with *n_estimators* = 4000
+ RandomForestClassifier
> With just default value this gave acc = 72.81%
> <br> Changed the *criterian* = "entropy" and *max_features* = None it gave the best result of accuracy __73.98%__
> <br> **Note** Everytime I ran this algorithm keeps changing the accuracy, may be because of randomness. 



In [246]:
# Generate cross validation folds for dataset.  It return the row indices corresponding to train and test.
# We set random_state to ensure we get the same splits every time we run this.

GE_Tkts_data = GE_Tkts[predictors]
GE_Tkts_targets = GE_Tkts["SLA Met"]

In [274]:
predictions = []
#for train, test in kf:
# The predictors we're using the train the algorithm.  Note how we only take the rows in the train folds.

X_train, X_test, y_train, y_test = train_test_split(GE_Tkts_data, GE_Tkts_targets, test_size=0.1, random_state=0)

# Training the algorithm using the predictors and target.
alg.fit(X_train, y_train)
# We can now make predictions on the test fold
test_predictions = alg.predict(X_test)
print(alg.score(X_test,y_test))

# i = 0
# sum = 0.00
# for SLA in y_test:
#     diff = age - test_predictions[i]
# #     print(diff, age, test_predictions[i])
#     sum = sum + np.square(diff)
#     i = i + 1
# avg = sum / y_test.shape[0]
# deviation = np.sqrt(avg)
# print(deviation)


0.726213592233


In [248]:
# X1_test = [[10,70,2]]
# print(alg.predict(X1_test))

index = 13
print(GE_Tkts_data.iloc[index])
test_predictions = alg.predict(GE_Tkts_data.iloc[index])

# test_predictions = alg.predict(X1_test)

print("Predicted Value", test_predictions)
print("Actual Value", GE_Tkts_targets.iloc[index])


Cat Code          8
Sub Cat code     60
Priority Code     3
Name: 13, dtype: int64
Predicted Value [1]
Actual Value 1


C:\Users\karthik.sunil\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\utils\validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
